<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

### 图像说明

在本实验中，我们会将深度学习工作流程与 AI 相结合，以生成对场景的说明。Microsoft Common Object in Context (MSCOCO) 数据集包含很多图像，每个图像都有 5 条说明，因此我们已得到输入和相应的期望输出，这两个用于监督式学习的必需要素。我们面临的挑战将是，在给定视觉数据和基于语言的数据的情况下确定正确的网络工作流程。
通过图像分类，我们学习了如何生成对图像的高层次理解。借助自然语言处理的基础知识，我们学习了如何使用递归神经网络 (RNN) 根据上下文生成文本。在本实验中，我们将学习向使用 RNN 生成语言预测的上下文中添加对图像的高层次理解。
我们从导入模块开始：


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import inspect
import time

import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
#import reader
import collections
import os
import re
import json

import matplotlib.pyplot as plt

from scipy import ndimage
from scipy import misc
import sys
sys.path.insert(0, '/dli/data/mdt/models/slim')

slim=tf.contrib.slim
from nets import vgg

from preprocessing import vgg_preprocessing

%matplotlib inline  
!nvidia-smi

### 像素到上下文

首先检验一种修改图像分类工作流程，以生成对图像（比简单标签保留的信息更多）的高层次理解的方法。
我们将通过 VGG 来输入 MSCOCO 中的图像，VGG 是一个屡获殊荣的图像分类网络。我们将不会查看 VGG“softmax”或预测层输出，而是查看在此之前的层。这是称为 fc7 的
 [特征向量](#fv "An array midway through the neural network that represents some features of the input")。

In [ ]:
TRAIN_IMAGE_PATH='/dli/data/mdt/mscoco/train2014/'
## Read Training files
with open("/dli/data/mdt/mscoco/captions_train2014.json") as data_file:
         data=json.load(data_file)

image_feature_vectors={}   
tf.reset_default_graph()
    
one_image=ndimage.imread(TRAIN_IMAGE_PATH+data["images"][0]['file_name'])
    #resize for vgg network
resize_img=misc.imresize(one_image,[224,224])
if len(one_image.shape)!= 3: #Check to see if the image is grayscale if True mirror colorband
    resize_img=np.asarray(np.dstack((resize_img, resize_img, resize_img)), dtype=np.uint8)

processed_image = vgg_preprocessing.preprocess_image(resize_img, 224, 224, is_training=False)
processed_images  = tf.expand_dims(processed_image, 0)      
network,endpts= vgg.vgg_16(processed_images, is_training=False)

   
init_fn = slim.assign_from_checkpoint_fn(os.path.join('/dli/data/mdt/mscoco/vgg_16.ckpt'),slim.get_model_variables('vgg_16'))
sess = tf.Session()
init_fn(sess)
NETWORK,ENDPTS=sess.run([network,endpts])
sess.close()
print('fc7 array for a single image')
print(ENDPTS['vgg_16/fc7'][0][0][0])  

很显然，其中包含很多信息。我们通过将该矢量可视化为一张图形来找出“...”中包含的内容。

In [ ]:
plt.plot(ENDPTS['vgg_16/fc7'][0][0][0])
plt.xlabel('feature vector index')
plt.ylabel('amplitude')
plt.title('fc7 feature vector')
data["images"][0]['file_name']

What you see here are more than 4000 parameters of varying amplitudes. Typically, the next layer of a Convolutional Neural Network (CNN) would simplify that to generate a label, such as "cat" or "dog". However, clearly, there is more to every image than the predominant object it contains.

- What kind of cat?  
- What is it doing?  
- What else is important?  
- Etc.  

虽然此图形对我们来说可能意义不大，但这是旨在从像素数据中抽取特征的神经网络的结果。我们会将此向量作为“上下文”，用于生成说明。

在执行此操作之前，可以通过对下方代码块的试验查看网络中不同层的响应。

如何查看第一个卷积层中的特征图？如果您需要 [提示](#answer1 "The output from the convolutional layer is in the form of height, width, and number of feature maps. FEATUREMAPID can be any value between 0 and the number of feature maps minus 1.")，请查看此处。

In [ ]:
featuremap='vgg_16/conv1/conv1_1'
print(ENDPTS[featuremap][0].shape)
FEATUREMAPID=63
print('input image and feature map from ',featuremap)
plt.subplot(1,2,1)
plt.imshow(resize_img)
plt.subplot(1,2,2)
plt.imshow(ENDPTS[featuremap][0][:,:,FEATUREMAPID])

如何查看网络中不同层的响应？

## Combining Images and Captions

### Load high-level feature vector from images

For the sake of time, we have run each image in the dataset through VGG and saved the feature vectors to a [numpy](https://www.numpy.org) file. Load them here.

In [ ]:
example_load=np.load('/dli/data/mdt/mscoco/train_vgg_16_fc7_2000.npy').tolist()
image_ids=example_load.keys()
print('Number of images in the training set: ', len(example_load.keys()))

### 将说明与图像匹配

接下来，我们会将特征图与其各自的说明相结合。很多图像都有 5 条说明。运行下面的代码以查看某个图像的说明。

In [ ]:
CaptionsForOneImage=[]
for k in range(len(data['annotations'])):
    if data['annotations'][k]['image_id']==data["images"][0]['id']:
        CaptionsForOneImage.append([data['annotations'][k]['caption'].lower()])
plt.imshow(resize_img)
print('MSCOCO captions for a single image')
CaptionsForOneImage

由于时间原因，我们已创建一个文件，其中包含来自 2000 张已创建的 MSCOCO 图像中的特征向量。接下来，您将加载这些图像并将其与说明匹配。请注意，运行此步骤可能需要花费 5 分钟以上的时间。在等待期间，可以花点时间了解 MSCOCO 数据集。主网站为 www.mscoco.org。

In [ ]:
#Create 3 lists image_id, feature maps, and captions.
image_id_key=[]
feature_maps_to_id=[]
caption_to_id=[]
for observed_image in image_ids:   
    for k in range(len(data['annotations'])):
        if data['annotations'][k]['image_id']==observed_image:
            image_id_key.append([observed_image])
            feature_maps_to_id.append(example_load[observed_image])
            caption_to_id.append(re.sub('[^A-Za-z0-9]+',' ',data['annotations'][k]['caption']).lower()) #remove punctuation 
  
print('number of images ',len(image_ids))
print('number of captions ',len(caption_to_id))

在上面的单元中，我们创建了三个列表，分别用于 image_id、特征图。和说明。要验证每个列表的索引是否已完成匹配，可以显示某个图像的图像 id 和说明。

In [ ]:
STRING='%012d' % image_id_key[0][0]
exp_image=ndimage.imread(TRAIN_IMAGE_PATH+'COCO_train2014_'+STRING+'.jpg')
plt.imshow(exp_image)
print('image_id ',image_id_key[:5])
print('the captions for this image ')
print(caption_to_id[:5])

由于我们已匹配图像 id、特征向量和说明，可以按照使用 RNN 预测下一个单词时的操作对说明进行整理以实施训练。

### Encode our captions and set a dictionary limit

When using one-hot encoding, the vector length of each word is equal to the number of words in the dictionary. Run the cell below to learn about the shape of your data.

In [ ]:
num_steps=20
######################################################################
##Create a list of all of the sentences.
DatasetWordList=[]
for dataset_caption in caption_to_id:
        DatasetWordList+=str(dataset_caption).split()
#Determine number of distinct words 
distinctwords=collections.Counter(DatasetWordList)
#Order words 
count_pairs = sorted(distinctwords.items(), key=lambda x: (-x[1], x[0])) #ascending order
words, occurence = list(zip(*count_pairs))
#DictionaryLength=occurence.index(4) #index for words that occur 4 times or less
words=['PAD','UNK','EOS']+list(words)#[:DictionaryLength])
word_to_id=dict(zip(words, range(len(words))))
#####################  Tokenize Sentence #######################
Tokenized=[]
for full_words in caption_to_id:
        EmbeddedSentence=[word_to_id[word] for word in full_words.split() if word in word_to_id]+[word_to_id['EOS']]
        #Pad sentences that are shorter than the number of steps 
        if len(EmbeddedSentence)<num_steps:
            b=[word_to_id['PAD']]*num_steps
            b[:len(EmbeddedSentence)]=EmbeddedSentence
        if len(EmbeddedSentence)>num_steps:
            b=EmbeddedSentence[:num_steps]
        if len(b)==EmbeddedSentence:
            b=EmeddedSentence
        #b=[word_to_id['UNK'] if x>=DictionaryLength else x for x in b] #turn all words used 4 times or less to 'UNK'
        #print(b)
        Tokenized+=[b]
        
print("Number of words in this dictionary ", len(words))

In [ ]:
#Tokenized Sentences
Tokenized[::2000]

下一个单元包含用于对我们的数据和 RNN 模型进行排列的函数。每个函数的输出结果应该是什么？如果您需要提示，请将鼠标指针悬停在[此处](#answer2 "The data_queue function batches the data for us, this needs to return tokenized_caption, input_feature_map. The RNN model should return prediction before the softmax is applied and is defined as pred.").

In [ ]:
def data_queue(caption_input,feature_vector,batch_size,):


    train_input_queue = tf.train.slice_input_producer(
                    [caption_input, np.asarray(feature_vector)],num_epochs=10000,
                                    shuffle=True) #False before

    ##Set our train data and label input shape for the queue

    TrainingInputs=train_input_queue[0]
    FeatureVectors=train_input_queue[1]
    TrainingInputs.set_shape([num_steps])
    FeatureVectors.set_shape([len(feature_vector[0])]) #fc7 is 4096
    min_after_dequeue=1000000
    capacity = min_after_dequeue + 3 * batch_size 
    #input_x, target_y
    tokenized_caption, input_feature_map = tf.train.batch([TrainingInputs, FeatureVectors],
                                                 batch_size=batch_size,
                                                 capacity=capacity,
                                                 num_threads=6)
    return ##FIXME##,##FIXME##
    
    

def rnn_model(Xconcat,input_keep_prob,output_keep_prob,num_layers,num_hidden):
#Create a multilayer RNN
#reuse=False for training but reuse=True for sharing
    layer_cell=[]
    for _ in range(num_layers):
        lstm_cell = tf.contrib.rnn.LSTMCell(num_units=num_hidden, state_is_tuple=True)
        lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell,
                                          input_keep_prob=input_keep_prob,
                                          output_keep_prob=output_keep_prob)
        layer_cell.append(lstm_cell)

    cell = tf.contrib.rnn.MultiRNNCell(layer_cell, state_is_tuple=True)
    outputs, last_states = tf.contrib.rnn.static_rnn(
        cell=cell,
        dtype=tf.float32,
        inputs=tf.unstack(Xconcat))

    output_reshape=tf.reshape(outputs, [batch_size*(num_steps),num_hidden]) #[12==batch_size*num_steps,num_hidden==12]
    pred=tf.matmul(output_reshape, variables_dict["weights_mscoco"]) +variables_dict["biases_mscoco"]
    return ##FIXME##

现在，我们要训练 RNN。我们将使用 [1]、[2] 和 [3] 中引用的论文中概述的方法：在此方法中的每个时间步长，我们会通过将每个新输入与特征向量相连接来向 RNN 输入图像的当前说明和特征向量。请随意查看任何论文，以便获得更深入的了解。

In [ ]:
tf.reset_default_graph()
#######################################################################################################
# Parameters
num_hidden=2048
num_steps=num_steps
dict_length=len(words)
batch_size=4
num_layers=2
train_lr=0.00001
#######################################################################################################
TrainingInputs=Tokenized
FeatureVectors=feature_maps_to_id

## Variables ## 
# Learning rate placeholder
lr = tf.placeholder(tf.float32, shape=[])
#tf.get_variable_scope().reuse_variables()

variables_dict = {
    "weights_mscoco":tf.Variable(tf.truncated_normal([num_hidden,dict_length],
                                                     stddev=1.0,dtype=tf.float32),name="weights_mscoco"),
    "biases_mscoco": tf.Variable(tf.truncated_normal([dict_length],
                                                     stddev=1.0,dtype=tf.float32), name="biases_mscoco")}


tokenized_caption, input_feature_map=data_queue(TrainingInputs,FeatureVectors,batch_size)
mscoco_dict=words

TrainInput=tf.constant(word_to_id['PAD'],shape=[batch_size,1],dtype=tf.int32)
#Pad the beginning of our caption. The first step now only has the image feature vector. Drop the last time step 
#to timesteps to 20
TrainInput=tf.concat([tf.constant(word_to_id['PAD'],shape=[batch_size,1],dtype=tf.int32),
                      tokenized_caption],1)[:,:-1]
X_one_hot=tf.nn.embedding_lookup(np.identity(dict_length), TrainInput) #[batch,num_steps,dictionary_length][2,6,7]
#ImageFeatureTensor=input_feature_map
Xconcat=tf.concat([input_feature_map+tf.zeros([num_steps,batch_size,4096]), 
                     tf.unstack(tf.to_float(X_one_hot),num_steps,1)],2)#[:num_steps,:,:]

pred=rnn_model(Xconcat,1.0,1.0,num_layers,num_hidden)


#the full caption is the target sentence
y_one_hot=tf.unstack(tf.nn.embedding_lookup(np.identity(dict_length), tokenized_caption),num_steps,1) #[batch,num_steps,dictionary_length][2,6,7]

y_target_reshape=tf.reshape(y_one_hot,[batch_size*num_steps,dict_length])


# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y_target_reshape))

optimizer = tf.train.MomentumOptimizer(lr,0.9)

gvs = optimizer.compute_gradients(cost,aggregation_method = tf.AggregationMethod.EXPERIMENTAL_TREE)
capped_gvs = [(tf.clip_by_value(grad, -10., 10.), var) for grad, var in gvs]
train_op=optimizer.apply_gradients(capped_gvs)

saver = tf.train.Saver()

init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())    

with tf.Session() as sess:
        
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #Load a pretrained network
    saver.restore(sess, '/dli/data/mdt/mscoco/rnn_layermodel_iter40000')
    print('Model restored from file')
    
    for i in range(100):
        
        loss,y_pred,target_caption,_=sess.run([cost,pred,tokenized_caption,train_op],feed_dict={lr:train_lr})

        if i% 10==0:
            print("iteration: ",i, "loss: ",loss)
            
    MODEL_NAME='/dli/data/mdt/mscoco/rnn_model_iter'+str(i)
    saver.save(sess, MODEL_NAME) 
    print('saved trained network ',MODEL_NAME)
    print("Done Training")
    coord.request_stop()
    coord.join(threads)
    sess.close() 


尝试添加一些 print 语句，以检查向 RNN 的输入大小。在合并特征向量与说明后，RNN 输入的形状是什么？将鼠标指针悬停在 [此处](#print "print(Xconcat.shape)")可获取提示。

我们已要求 TensorFlow 按照网络最后一层的“带分对数的交叉熵”测量损失。这种损失方法可以让网络进行学习，并且我们可以将其性能与其他解决方案进行比较，但是您可能更感兴趣的是，与我们的预期相比，当前的解决方案是否有效。

我们可以借助以下函数使用上个批次的索引来评估该批次中的单个图像及其说明。如果您需要提示，请将鼠标指针悬停在[此处](#answer3 "if the batch_size is 4, batch_id may be any value between 0 and 3.")。

##### 请注意，根据保存神经网络时该网络的状态，可能会生成不完整、不连贯的说明，有时还可能生成不恰当的说明。

In [ ]:
def show_next_predicted_word(batch_id,batch_size,id_of_image,target_caption,predicted_caption,words,PATH):
        Target=[words[ind] for ind in target_caption[batch_id]]
        Prediction_Tokenized=np.argmax(predicted_caption[batch_id::batch_size],1)
        Prediction=[words[ind] for ind in Prediction_Tokenized]
        STRING2='%012d' % id_of_image
        img=ndimage.imread(PATH+STRING2+'.jpg')
        return Target,Prediction,img,STRING2

#You can change the batch id to a number between [0 , batch_size-1]
batch_id=##FIXME##
image_id_for_predicted_caption=[x for x in range(len(Tokenized)) if target_caption[batch_id].tolist()== Tokenized[x]][0]


t,p,input_img,string_out=show_next_predicted_word(batch_id,batch_size,image_id_key[image_id_for_predicted_caption][0]
                                         ,target_caption,y_pred,words,TRAIN_IMAGE_PATH+'COCO_train2014_')
print('Caption')
print(t)
print('Predicted Words')
print(p)
plt.imshow(input_img)


到目前为止，这是一个“下一单词”预测变量，通过添加图像作为上下文。您是否可以想象采用这一预测变量的部署场景？如果可以，这表示它很有效。但是，它仍然未解决我们着手应对的挑战：

**我们是否可以生成对这些场景的说明？**

让我们加载已保存的网络并使用其根据验证图像生成说明：

验证图像存储在 /data/mscoco/val2014 中。特征向量的 npy 文件储存在 /data/mscoco/val_vgg_16_fc7_100.npy。Add it using the np.load function below by replacing the ```'##FIXME##'``` with ```'/dli/data/mdt/mscoco/val_vgg_16_fc7_100.npy'```.

In [ ]:
##Load and test our test set
val_load=np.load('##FIXME##').tolist()
val_ids=val_load.keys()

#Create 3 lists image_id, feature maps, and captions.
val_id_key=[]
val_map_to_id=[]
val_caption_to_id=[]
for observed_image in val_ids:   
    val_id_key.append([observed_image])
    val_map_to_id.append(val_load[observed_image])
    
print('number of images ',len(val_ids))
print('number of captions ',len(val_map_to_id))

以下单元**仅**加载来自验证数据集中某张图像的特征向量，并将其与我们的预训练网络一起用于生成说明。使用 VALDATA 变量传播某张图像，使其通过 RNN，然后生成说明。您还需要加载刚才在训练中创建的网络。如果您需要提示，请查看[此处](#answer5 "Any of the of the data points in our validation set can be used here. There are 501 captions. Any number between 0 and 100-1 can be used for the VALDATA parameter, such as VALDATA=10.)")。

##### 请注意，根据保存神经网络时该网络的状态，可能会生成不完整、不连贯的说明，有时还可能生成不恰当的说明。

In [ ]:
tf.reset_default_graph()
batch_size=1
num_steps=20
print_topn=0 #0for do not display 
printnum0f=3
#Choose a image to caption
VALDATA=##FIXME##  #ValImage fc7 feature vector

variables_dict = {
    "weights_mscoco":tf.Variable(tf.truncated_normal([num_hidden,dict_length],
                                                     stddev=1.0,dtype=tf.float32),name="weights_mscoco"),
    "biases_mscoco": tf.Variable(tf.truncated_normal([dict_length],
                                                     stddev=1.0,dtype=tf.float32), name="biases_mscoco")}


StartCaption=np.zeros([batch_size,num_steps],dtype=np.int32).tolist()

CaptionPlaceHolder = tf.placeholder(dtype=tf.int32, shape=(batch_size , num_steps))

ValFeatureMap=val_map_to_id[VALDATA]
X_one_hot=tf.nn.embedding_lookup(np.identity(dict_length), CaptionPlaceHolder) #[batch,num_steps,dictionary_length][2,6,7]
    #ImageFeatureTensor=input_feature_map
Xconcat=tf.concat([ValFeatureMap+tf.zeros([num_steps,batch_size,4096]), 
                            tf.unstack(tf.to_float(X_one_hot),num_steps,1)],2)#[:num_steps,:,:]

pred=rnn_model(Xconcat,1.0,1.0,num_layers,num_hidden)
pred=tf.nn.softmax(pred)
saver = tf.train.Saver()

init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())    

with tf.Session() as sess:
        
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #Load a pretrained network
    saver.restore(sess, '##FIXME##')
    print('Model restored from file')
    for i in range(num_steps-1):
        predict_next_word=sess.run([pred],feed_dict={CaptionPlaceHolder:StartCaption})
        INDEX=np.argmax(predict_next_word[0][i])
        StartCaption[0][i+1]=INDEX
        ##Post N most probable next words at each step
        if print_topn !=0:
            print("Top ",str(printnum0f), "predictions for the", str(i+1), "word in the predicted caption" )
            result_args = np.argsort(predict_next_word[0][i])[-printnum0f:][::-1]
            NextWord=[words[x] for x in result_args]
            print(NextWord)
        
    coord.request_stop()
    coord.join(threads)
    sess.close() 

STRING2='%012d' % val_id_key[VALDATA][0]
img=ndimage.imread('/dli/data/mdt/mscoco/val2014/COCO_val2014_'+STRING2+'.jpg')
plt.imshow(img)
plt.title('COCO_val2014_'+STRING2+'.jpg')
PredictedCaption=[words[x] for x in StartCaption[0]]
print("predicted sentence: ",PredictedCaption[1:])

根据您所选择的图像，会获得不同水准的结果。还有其他方法可以改进结果，例如，使用更多的训练次数、更大/更多样的数据集、拟合度更高的数据网络，等。现在我们已经理解此项工作流程，我们将结束本次实验，并继续解决下一项挑战：如何为视频创建说明？

We'll introduce the difference, but then a lot of the workflow will be identical, so you will attempt to recreate it using this notebook and what you have already learned about Recurrent Neural Networks as a model. Run the cell below to make room in memory, which will give you an expected *Dead Kernel* error. 

In [ ]:
#Free our GPU memory before proceeding to the next part of the lab
import os
os._exit(00)

Then open the next notebook, [Video Captioning Challenge](../../task-assessment/task/VideoCaptioning-zh.ipynb)

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

## 参考资料

[1] Donahue, J, et al. "Long-term recurrent convolutional networks for visual recognition and description."     Proceedings of the IEEE conference on computer vision and pattern recognition. 2015.

[2]Vinyals, Oriol, et al. "Show and tell: Lessons learned from the 2015 mscoco image captioning challenge." IEEE transactions on pattern analysis and machine intelligence 39.4 (2017): 652-663.

[3] TensorFlow Show and Tell:A Neural Image Caption Generator [example] (https://github.com/tensorflow/models/tree/master/im2txt)

[4] Karapthy, A. [NeuralTalk2](https://github.com/karpathy/neuraltalk2)

[5]Lin, Tsung-Yi, et al. "Microsoft coco: Common objects in context." European Conference on Computer Vision. Springer International Publishing, 2014.